In [1]:
import os
import gc
import time
import random
from tqdm import tqdm

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models, datasets
from torchvision.transforms.functional import InterpolationMode
from torchvision.models.segmentation.fcn import FCNHead
from torchvision.models.segmentation.deeplabv3 import DeepLabV3, DeepLabHead
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
from torch.utils.data.dataloader import default_collate
from torchvision.models.segmentation import deeplabv3_resnet101
from PIL import Image
import imageio.v2 as imageio

/home/guest/r11922a05/anaconda3/envs/DLCV/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
tmp = models.vgg16(weights='IMAGENET1K_V1')

In [6]:
tmp

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [2]:
SEED = 5566
use_gpu = torch.cuda.is_available()
device = torch.device("cuda" if use_gpu else "cpu")
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
random.seed(SEED)
np.random.seed(SEED)

In [3]:
IMG_FILE = 'hw1_data/p2_data'
TRAIN_PATH = 'hw1_data/p2_data/train'
VAL_PATH = 'hw1_data/p2_data/validation'
TRAIN_NUM_PIC = 2000
VAL_NUM_PIC = 257
NUM_CLASS = 7

# Dataset

In [4]:
class P2Dataset(Dataset):
    def __init__(self, path, transform=None):
        self.path = path
        self.data = None #3x512x512
        self.labels = None
        self.load_image_p2(path)
        self.transform = transform

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img = self.read_img(idx)
        # img = torch.from_numpy(img).type(torch.LongTensor)
        label = self.read_mask(idx)
        label = torch.from_numpy(label)
        # label = torch.from_numpy(label).type(torch.LongTensor)
        return img, label

    def read_img(self, idx):
        path = os.path.join(self.path, self.data[idx])
        img = imageio.imread(path)
        if self.transform is not None:
            img = self.transform(img)
        return img

    def read_mask(self, idx):
        path = os.path.join(self.path, self.labels[idx])
        mask = imageio.imread(path) #512x512x3
        masks = np.empty((mask.shape[0],mask.shape[1]))

        #load mask
        mask = (mask >= 128).astype(int)
        #512x512x3 -> 512x512
        mask = 4 * mask[:, :, 0] + 2 * mask[:, :, 1] + mask[:, :, 2]
        masks[mask == 3] = 0  # (Cyan: 011) Urban land 
        masks[mask == 6] = 1  # (Yellow: 110) Agriculture land 
        masks[mask == 5] = 2  # (Purple: 101) Rangeland 
        masks[mask == 2] = 3  # (Green: 010) Forest land 
        masks[mask == 1] = 4  # (Blue: 001) Water 
        masks[mask == 7] = 5  # (White: 111) Barren land 
        masks[mask == 0] = 6  # (Black: 000) Unknown 
        return masks

    def load_image_p2(self, path):
        data, labels = [], []
        path = os.listdir(path)
        path = sorted(path)
        for img_name in path:
            file_name = img_name.split('.')
            #data
            if file_name[1] == 'jpg':
                data.append(img_name)
            #label
            elif file_name[1] == 'png':
                labels.append(img_name)
        self.data = data
        self.labels = labels

In [ ]:
class P2_TransformsModelA:
    def __init__(self):
        self.normalize = transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
            )
        
        self.transform = {
            'train': transforms.Compose([
                # transforms.RandomResizedCrop(224),
                transforms.ToTensor(),
                self.normalize,
            ]),
            'val':  transforms.Compose([
                # transforms.Resize(224),
                transforms.ToTensor(),
                self.normalize,
            ])
        }

In [11]:
customized_transform = P2_TransformsModelA()

train_dataset = P2Dataset(
    TRAIN_PATH, transform=customized_transform.transform['train']
    )
val_dataset = P2Dataset(
    VAL_PATH,  transform=customized_transform.transform['val']
    )

batch_size = 12
train_loader = DataLoader(
    train_dataset, batch_size=batch_size, 
    )
val_loader = DataLoader(
    val_dataset, batch_size=batch_size,
) 

# Model

## VGG16-FCN32

In [9]:
class VGG16_FCN32(torch.nn.Module):
    def __init__(self, n_classes):
        super(VGG16_FCN32, self).__init__()
        self.features = models.vgg16(weights='IMAGENET1K_V1').features 
        self.fc6 = nn.Sequential(
            nn.Conv2d(512, 4096, 2), #kernel = 7x7?
            nn.ReLU(inplace=True),
            nn.Dropout() #dropout2d?
        )
        self.fc7 = nn.Sequential(
            nn.Conv2d(4096, 4096, 1),
            nn.ReLU(inplace=True),
            nn.Dropout() #dropout2d?
        )
        self.clf = nn.Conv2d(4096, n_classes, 1)
        self.upconv = nn.ConvTranspose2d(
            n_classes, n_classes, kernel_size=64, stride=32, bias=False
            )

    def forward(self, x): #3x512x512
        x = self.features(x) #512x8x8
        x = self.fc6(x) #4096x7x7
        x = self.fc7(x) #4096x7x7
        x = self.clf (x) #7x7x7
        x = self.upconv(x) #7x512x512 (upsample 32x)
        #print(x.size())
        return x

In [ ]:
def train_P2(train_loader, model, loss_fn, optimizer, device):
    train_loss = []
    train_iou = []
    model.train()
    for img, label in tqdm(train_loader, leave=False, colour='green'):
        
        img, label = img.to(device), label.to(device)

        optimizer.zero_grad()
        output = model(img)

        loss = loss_fn(output, label) 
        loss.backward()            
        optimizer.step()
        
        with torch.no_grad():
            predict = torch.argmax(output, dim=1)
            predict = predict.detach().cpu().numpy() #numpy array
            label = label.detach().cpu().numpy() 
            iou = mean_iou_score(predict, label)
            loss = loss.item() #python float
            train_iou.append(iou)
            train_loss.append(loss) 
        
        del img, label, output, loss, predict, iou
        
    train_iou = np.mean(train_iou)
    train_loss = np.mean(train_loss)
    return train_iou, train_loss


def valid_P2(valid_loader, model, loss_fn, device):
    model.eval()
    with torch.no_grad():
        valid_loss = []
        valid_iou = []
        for img, label in valid_loader:
            img, label = img.to(device), label.to(device)
            
            output = model(img) 
            loss = loss_fn(output, label)
            
            predict = torch.argmax(output, dim=1)
            predict = predict.detach().cpu().numpy()
            label = label.detach().cpu().numpy()

            iou = mean_iou_score(predict, label)

            loss = loss.item()
            valid_loss.append(loss)
            valid_iou.append(iou)
            del img, label, output, loss, predict, iou
            

    valid_iou = np.mean(valid_iou)
    valid_loss = np.mean(valid_loss)
    return valid_iou, valid_loss

In [12]:
modelA = VGG16_FCN32(NUM_CLASS)
modelA = modelA.to(device)
optimizer = optim.SGD(
    modelA.parameters(),lr=0.0001,
    momentum=0.9
    )
#input: [0.5, 0.2, 0.6] 
#target: [1, 0, 1]
loss_fn = nn.CrossEntropyLoss() 

num_epoch = 200

In [27]:
loss_fn = nn.CrossEntropyLoss()
#loss_fn = FocalLoss(NUM_CLASS)
loss_fn = loss_fn.to(device)

lrs = [0.0001, 0.00005, 0.00001]
weight_decays = [0.001, 0.0001, 0.00001]
for lr in lrs:
    for weight_decay in weight_decays:
        print(f"lr = {lr}, weight_decay = {weight_decay}")
        model = VGG16_FCN32(NUM_CLASS)
        model = model.to(device)
        optimizer = optim.Adam(
            model.parameters(),
            lr=lr,
            weight_decay=weight_decay
            )   
        #input: [0.5, 0.2, 0.6] 
        #target: [1, 0, 1]

        num_epoch = 200

        iou_record = []
        best_valid_iou = 0
        best_valid_loss = 999999
        best_epoch = 0
        no_update_cnt = 0
        best_model = None
        patience = 5

        x = time.time()
        for epoch in range(num_epoch):
            train_iou, train_loss = train_P2(train_loader, model, loss_fn, optimizer, device)
            valid_iou, valid_loss = valid_P2(val_loader, model, loss_fn, device)
            iou_record.append(valid_iou)

            if valid_iou > best_valid_iou:
                best_valid_loss = valid_loss
                best_valid_iou = valid_iou
                best_epoch = epoch
                no_update_cnt = 0
                best_model = model

            else:
                no_update_cnt += 1
            y = time.time()
            time_train = (y - x) / 60
            print(f"Epoch {epoch+1}: {round(time_train, 2)} min elapsed, train iou: {round(train_iou * 100, 2)}%, train loss: {round(train_loss, 3)}, valid iou: {round(valid_iou * 100, 2)}%, valid loss: {round(valid_loss, 3)}")
            #Early stop
            if no_update_cnt > patience:
                break
        model_name = f"models/p2/1003_VGG16FCN32_lr_{lr}_weight_decay_{weight_decay}"
        save_checkpoint(
            best_model, optimizer, 
            round(best_valid_iou, 2), prefix=model_name
        )  
        print('########################################################')
        print("Finish model tuning")
        print(f"Best epoch is {best_epoch}, Iou: {best_valid_iou}, Loss: {best_valid_loss}")
        print('########################################################')

lr = 0.0001, weight_decay = 0.001


Epoch 1: 2.96 min elapsed, train iou: 13.35%, train loss: 1.105, valid iou: 23.21%, valid loss: 0.862


Epoch 2: 5.93 min elapsed, train iou: 32.92%, train loss: 0.742, valid iou: 39.04%, valid loss: 0.708


Epoch 3: 8.91 min elapsed, train iou: 43.96%, train loss: 0.622, valid iou: 43.76%, valid loss: 0.689


Epoch 4: 11.9 min elapsed, train iou: 45.35%, train loss: 0.604, valid iou: 44.3%, valid loss: 0.681


...

Epoch 6: 17.82 min elapsed, train iou: 52.18%, train loss: 0.463, valid iou: 52.46%, valid loss: 0.469


Epoch 7: 20.79 min elapsed, train iou: 54.79%, train loss: 0.435, valid iou: 54.06%, valid loss: 0.475


Epoch 8: 23.75 min elapsed, train iou: 56.94%, train loss: 0.398, valid iou: 54.83%, valid loss: 0.467


Epoch 9: 26.71 min elapsed, train iou: 58.03%, train loss: 0.381, valid iou: 51.54%, valid loss: 0.534


 37%|███▋      | 61/167 [01:00<01:45,  1.01it/s]